# Movie Recommendation System

© Explore Data Science Academy

---
### Honour Code

I {**2307FTDS_team_NM3 Members**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Twitter Sentiment Classification

In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

With this context, EDSA is challenging you to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being exposed to content they would like to view or purchase - generating revenue and platform affinity.

2307FTDS_team_NM3 Members:
 1.
 2.
 3.
 4.
 5. Christelle Coetzee
 6.

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Introduction</a>

<a href=#two>2. Importing Packages</a>

<a href=#three>3. Loading Data</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Feature Engineering</a>

<a href=#six>6. Modeling</a>

<a href=#seven>7. Model Performance</a>

<a href=#eight>8. Model Explanations</a>

<a href=#nine>9. Conclusion</a>

 <a id="one"></a>
## 1. Introduction
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Introduction ⚡ |
| :--------------------------- |
| In this section, we will introduce our institution and provide our problem statement. |

<a id="two"></a>
## 2. Import Packages and Libaries
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: package importation⚡ |
| :--------------------------- |
| In this section we will import all the packages we will need in order to run our notebook. |

---

In [1]:
# Installations:

# Uncomment and run the following commands to install necessary libraries:

# Install Comet ML for experiment tracking
#!pip install comet_ml

# Install WordCloud for generating word clouds
#!pip install wordcloud

# Install Contractions for handling contractions in text data
#!pip install contractions

# Install CatBoost for CatBoost Classifier
#!pip install catboost

# Install XGBoost for XGBoost Classifier
#!pip install xgboost

# Install MLxtend for additional machine learning utilities
#!pip install mlxtend

# Install or upgrade TensorFlow for neural network models
#!pip install --upgrade tensorflow

In [24]:
# Import necessary libraries
import numpy as np  # Numerical operations library
import pandas as pd  # Data manipulation and analysis library
import matplotlib.pyplot as plt  # Plotting library
import seaborn as sns  # Data visualization library based on Matplotlib
%matplotlib inline
from wordcloud import WordCloud  # Word cloud generation library
import sweetviz as sv

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import Comet ML for experiment tracking
from comet_ml import Experiment  # Experiment tracking library

# Import Natural Language Toolkit (nltk) for text processing
import nltk  # Natural Language Toolkit
from nltk.corpus import stopwords  # Stopwords for text processing
from nltk.stem import WordNetLemmatizer  # Lemmatization for text processing
from nltk.tokenize import word_tokenize, TreebankWordTokenizer  # Tokenization for text processing
nltk.download('punkt')
nltk.download('wordnet')
import unicodedata  # Unicode character normalization
import contractions  # Expansion of contractions in text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # Text vectorization libraries

# Import regular expressions and string manipulation module
import re  # Regular expressions
import string  # String manipulation
import itertools  # Iteration tools
from collections import Counter  # Collection data type

# Libraries for data preparation and model building
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix  # Classification metrics
from sklearn.model_selection import train_test_split, GridSearchCV  # Train-test split and grid search
from sklearn.linear_model import LogisticRegression  # Logistic Regression classifier for machine learning
from sklearn.tree import DecisionTreeClassifier  # Decision Tree classifier for machine learning
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier for machine learning
from sklearn.svm import LinearSVC, SVC  # Support Vector Machine classifiers
from sklearn.naive_bayes import GaussianNB, MultinomialNB  # Naive Bayes classifiers
from sklearn.ensemble import BaggingClassifier  # Bagging classifier
from sklearn.ensemble import ExtraTreesClassifier  # Extra Trees classifier
from sklearn.ensemble import VotingClassifier  # Voting classifier
from xgboost import XGBClassifier  # Efficient and flexible gradient boosting library
from catboost import CatBoostClassifier  # High-performance gradient boosting on decision trees library
from scipy.sparse import hstack  # Used for stacking sparse matrices horizontally
import pickle  # Serialization library
from sklearn.utils import resample  # Resampling tool
from sklearn import feature_selection  # Feature selection module
from sklearn.feature_selection import f_classif  # Feature selection using F-statistic
from mlxtend.feature_selection import SequentialFeatureSelector  # Sequential feature selection
from sklearn import preprocessing  # Data preprocessing
import pickle  # Serialization library
import tensorflow as tf  # TensorFlow for neural net
from tensorflow.keras.layers import Dense  # Dense layer in Keras
from tensorflow.keras.models import Sequential  # Sequential model in Keras
from tensorflow.keras.utils import to_categorical  # Conversion to categorical data

# Feature selection Libraries:
from sklearn.feature_selection import SelectKBest  # To reduce features
from sklearn.feature_selection import chi2  # Used to estimate which features are most impactful

# Global Constants for reproducibility and consistency
TRAIN_TEST_SPLIT_VAR = 0.2  # Ratio for train-test split
RAND_STATE = 42  # Random seed for reproducibility
MAX_TEXT_FEATURES = 5000  # Maximum number of text features for all algorithms with exceptions listed below
RAD_SVC_Text_Features = 1000  # Specifically for use in radial SVC
NN_Text_Features = 1000  # Specifically for use in neural net
XGB_Text_Features = 1000  # Specifically for use in XGBoost algorithm
EXTRA_Text_Features = 1000  # Specifically for use in Extra Trees algorithm
BAG_Text_Features = 1000  # Specifically for use in Bagging algorithm
VEC_MIN_WORD_TO_REMOVE = 3  # Remove words that occur less than this value in the dataset

# Flags for notebook Execution
COMET_FLAG = False  # To gauge whether to commit experiments to Comet ML
VECTORIZER_TO_USE = "tfidf"  # Chooses between TfIDF vectorizer or Count Vectorizer - accepted values are "tfidf" or "count"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T460\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\T460\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<a id="three"></a>
## 3. Loading of Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: data loading ⚡ |
| :--------------------------- |
| In this section, we will load our datasets. |

---

Let's begin by loading our data into a DataFrame:

In [5]:
# Read training data from CSV file into a DataFrame
df_genome_scores = pd.read_csv('genome_scores.csv')

df_genome_tags = pd.read_csv('genome_tags.csv')

df_imdb_data = pd.read_csv('imdb_data.csv')

df_links = pd.read_csv('links.csv')

df_movies = pd.read_csv('movies.csv')

df_tags = pd.read_csv('tags.csv')

df_train = pd.read_csv('train.csv')

# Read test from CSV file into a DataFrame
df_test = pd.read_csv('test.csv')

<a id="four"></a>
## 4. Exploratory Data Analysis
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: EDA process ⚡ |
| :--------------------------- |
| In this section, we will perform an in-depth analysis of all the variables in the DataFrame. |

### **4.1. Data Overview:**

In [4]:
df_genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [14]:
df_genome_scores.shape

(15584448, 3)

In [26]:
analyze_report = sv.analyze(df_genome_scores)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [34]:
df_genome_tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [15]:
df_genome_tags.shape

(1128, 2)

In [27]:
analyze_report = sv.analyze(df_genome_tags)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [7]:
df_imdb_data.head()

,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [16]:
df_imdb_data.shape

(27278, 6)

In [28]:
analyze_report = sv.analyze(df_imdb_data)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [8]:
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [17]:
df_links.shape

(62423, 3)

In [29]:
analyze_report = sv.analyze(df_links)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [9]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [18]:
df_movies.shape

(62423, 3)

In [30]:
analyze_report = sv.analyze(df_movies)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [19]:
df_tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [20]:
df_tags.shape

(1093360, 4)

In [31]:
analyze_report = sv.analyze(df_tags)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [11]:
df_train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [21]:
df_train.shape

(10000038, 4)

In [32]:
analyze_report = sv.analyze(df_train)
analyze_report.show_html('report.html', open_browser=True)

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
ERROR: comet_ml is installed, but not configured properly (e.g. check API key setup). HTML reports will not be uploaded.


In [12]:
df_test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [22]:
df_test.shape

(5000019, 2)

In [33]:
# Merge genome scores with tags
merged_genome = pd.merge(df_genome_scores, df_genome_tags, on='tagId', how='inner')

# Merge IMDb data with movies
merged_imdb_movies = pd.merge(df_imdb_data, df_movies, on='movieId', how='inner')

# Merge links with the combined dataset
final_dataset = pd.merge(merged_imdb_movies, df_links, on='movieId', how='inner')

# Display the first few rows of the final dataset
final_dataset.head()

,movieId,title_cast,director,runtime,budget,plot_keywords,title,genres,imdbId,tmdbId
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion,Father of the Bride Part II (1995),Comedy,113041,11862.0


<a id="five"></a>
## 5. Feature Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section we will clean the dataset, and create new features - as identified in the EDA phase. |

---

<a id="six"></a>
## 6. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, we will create models that are able to accurately recommend movies to users. |

---

<a id="seven"></a>
## 7. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section we will compare the relative performance of the various trained machine learning models on a holdout dataset and comment on what model is the best and why. |

---

<a id="eight"></a>
## 8. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we will compare the models and determine the best model to use. We will also discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

<a id="nine"></a>
## 9. Conclusion
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we will conclude the project. |

---